# Доклад про Sympy

![picture](banner.svg)

**SymPy** - библиотека для символьных вычислений. Главная идея: мы работаем не с числами как, например, в NumPy, а с формулами как объектами. 

Мы использовали уже эту библиотеку в лабораторных работах по математическому анализу на 1 и 2 курсе. В отличии от NumPy, после подсчета мы получаем не просто числа, а какое то выражение как объект. С помощью SymPy мы можем его упростить, взять производную, предел, разложить в ряд и т.д.

### Установка: 

Установить можно с  https://pypi.python.org/pypi/sympy/ или просто

In [ ]:
%pip install sympy

Чтобы установисть используя Anaconda:

In [40]:
# %conda install -c anaconda sympy

Чтобы установить SymPy из исходного кода GitHub, сначала клонируйте SymPy с помощью git:

In [41]:
# %git clone https://github.com/sympy/sympy.git

Затем в клонированном вами репозитории sympy просто запустить:

In [42]:
# %pip install .

### Документация и использование 

Документация: https://docs.sympy.org/

Подробные инструкции по установке и созданию документации приведены в руководстве по стилю документации SymPy: https://docs.sympy.org/dev/contributing/documentation-style-guide.html

#### Символы

Символы, это объекты которые мы исподьзуем для вычисления и формул и подстановки в выражения

In [43]:
from sympy import symbols
x, y = symbols('x y')

#### Выражения

Теперь подставим в какое нибудь выраженение

In [44]:
expr = (x**2 - 1)/(x - 1)
expr

(x**2 - 1)/(x - 1)

Можем подставить в наш объект expr конткретное число

In [45]:
expr.subs(x, 3)

4

Или упростить

In [46]:
from sympy import simplify
simplify(expr)

x + 1

Так же можем просто раскрыть скобки

In [47]:
from sympy import expand
expand((x + 5) ** 2)

x**2 + 10*x + 25

Или наоборот факторизовать

In [48]:
from sympy import factor
factor(x**2 + 10 * x + 25)

(x + 5)**2

вывод в ввиде формулы латех:

In [49]:
from sympy import latex, pretty

expr = (x + 1) ** 2
latex(expr)

'\\left(x + 1\\right)^{2}'

текстовое форматирование:

In [50]:
pretty(expr)

'       2\n(x + 1) '

#### Приложения в математическом анализе

Мы уже использовали эту библиотеку в лабораторных по математическому анализу. Давайте вспомним все еще раз

##### Пределы

In [51]:
from sympy import limit, oo
expr = (1 + 1 / x) ** x
limit(expr, x, oo) #выражение, переменная, к чему стремится. в нашем случае oo - inf

E

##### Производные

In [52]:
from sympy import cos, exp, diff

expr = cos(x) * exp(x)
diff(expr)

-exp(x)*sin(x) + exp(x)*cos(x)

##### Интегралы

In [53]:
from sympy import integrate
integrate(expr)

exp(x)*sin(x)/2 + exp(x)*cos(x)/2

##### Разложение в Ряд Тейлора

In [54]:
from sympy import sin
expr = sin(x)
expr.series(x, 0, 6) #по какой переменной, точка разложения, точность

x - x**3/6 + x**5/120 + O(x**6)

#### Приложения в линейной алгебре

##### Решение уравнений и систем уравнений

In [55]:
from sympy import Eq, solve
# x^2 - 2 = 0
solve(Eq(x**2 - 2, 0), x)

[-sqrt(2), sqrt(2)]

In [56]:
# x + y = 3, x - y = 1
solve([Eq(x + y, 3), Eq(x - y, 1)], [x, y])

{x: 2, y: 1}

##### Работа с Матрицами (Слава Матричному Богу!)

In [57]:
from sympy import Matrix

M = Matrix([[1, 2],
            [3, 4]])

M.det()

-2

In [58]:
M.inv()

Matrix([
[ -2,    1],
[3/2, -1/2]])

In [59]:
M.eigenvals()

{5/2 - sqrt(33)/2: 1, 5/2 + sqrt(33)/2: 1}

##### Численные выражения

In [60]:
import numpy as np
from sympy import lambdify

expr = sin(x) / x

#lambdify(variables, expression, modules=...)
f = lambdify(x, expr, 'numpy') #превратили в численную функцию 

f(np.array([1.0, 0.1, 0.001]))

array([0.84147098, 0.99833417, 0.99999983])

### Техническая реализация

#### Древовидная структура

Мы посмотрели примеры работы, теперь давайте посмотрим как это устроено внутри.

Любое выражение в SymPy это дерево, где узлы это объекты SumPy - операции (+, *, **) и функции, листья - символы (Symbol), числа (Integer, Rational, Float), константы (pi, e)

Давайте посмотрим подробнее:

In [61]:
from sympy import srepr
expr = sin(x + 2*y)
srepr(expr)

"sin(Add(Symbol('x'), Mul(Integer(2), Symbol('y'))))"

Можем увидет узлы и листья выражения:

In [62]:
def print_tree(expr, l=0):
    print("  " * l + f"{type(expr).__name__}: {expr}")
    for arg in expr.args:
        print_tree(arg, l + 1)

print_tree(expr)

sin: sin(x + 2*y)
  Add: x + 2*y
    Symbol: x
    Mul: 2*y
      Integer: 2
      Symbol: y


          sin
           |
          Add
         /   \
        x    Mul
            /   \
           2     y


#### Базовый класс Basic

Почти все в SymPy наследуется от класса Basic:

     Basic
     ├── Atom
     │    ├── Symbol
     │    ├── Integer, Rational, Float
     │    └── NumberSymbol (pi, E, ...)
     └── Expr
          ├── Add, Mul, Pow
          ├── Function
          └── ...


Basic обеспечивает единое поведение для всех объектов: 
1. как хранить аргументы (args)
2. как сравнивать (\_\_eq\_\_)
3. как хэшировать(\_\_hash\_\_)
и т.д.

Поэтому любой объект однозначно задается парой expr.func, expr.args, порядок args не важен

In [63]:
a = x + 1
b = 1 + x
a == b

True

In [64]:
print(a.args, b.args)
print(a.args == b.args)

(1, x) (1, x)
True


In [65]:
print(a.func, b.func)
print(a.func == b.func)

<class 'sympy.core.add.Add'> <class 'sympy.core.add.Add'>
True


После создания объекты неизмемяемы

In [66]:
a.args = (2, x)

AttributeError: property 'args' of 'Add' object has no setter

#### Хэширование

Хэширование нужно для корректной работы set, dict, чтоб в деревьях не было копий и т.д.

hash(expr) = hash(expr.func, expr.args). a == b $\implies$ hash(a) == hash(b).

Объект в SymPy не изменяем, поэтому хэш тоже

В классе Basic это реализовано примерно так:

In [ ]:
class Basic:
    def __hash__(self):
        if self._hash is None:
            self._hash = hash((self.func, self.args))
        return self._hash

In [ ]:
print(a == b)
print(hash(a) == hash(b))
print(hash(x + 1) == hash(x + 2))

True
True
False


#### Решение уравнений

SymPy не использует один универсальный алгоритм для всех уравнений. На самом деле внутри происходит, что то такого:

1. $lhs == rhs \implies f(x) = lhs - rhs = 0$
2. Перед реальным решением выражение максимально упрощается (factor, expand, вынос общих множетелей, перенос слагаемых и т.д.). Это помогает свести задачу к более простой форме.
3. Далее SymPy по структуре $f(x)$ решает какие алгоритмы использовать (полиномиальные, экспоненциальные преобразования, тригономертрические тождества, системы и подстановки и т.д.). Используются алгоритмы из компьютерной алгебры.
4. Для решения систем линейных уравнений, внутри используются матричные методы (метод Гаусса, Matrix.solve, linsolve, разложения матриц). Для нелинейных используются подстановка, разложения на множители, комбинация символических решений и численных методов.
5. Если аналитически решить нельзя или пользователь явно просит численное решение, используюся функции nroots(численные корни полиномов), nsovle(обобщённые численные методы, например, Ньютона). Но тогда решение выдаётся приближённо, а не в виде точных формул.
6. После нахождения кандидатов SymPy подставляет их в исходное уравнение, отбрасыват посторонние корни и учитывает ограничения